In [ ]:
# Import required libraries
import os
import gc
import sys
import json
import random
from pathlib import Path

import cv2 # CV2 for image manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
max

In [ ]:
!ls /kaggle/input/imaterialist-fashion-2020-fgvc7/

In [ ]:
%%time
with open('/kaggle/input/imaterialist-fashion-2020-fgvc7/label_descriptions.json', 'r') as file:
    label_desc = json.load(file)
sample_sub_df = pd.read_csv('/kaggle/input/imaterialist-fashion-2020-fgvc7/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/imaterialist-fashion-2020-fgvc7/train.csv')

In [ ]:
train_df.head()

In [ ]:
sample_sub_df.head()

In [ ]:
print(f'Shape of training dataset: {train_df.shape}')

In [ ]:
print(f'# of images in training set: {train_df["ImageId"].nunique()}')
print(f'# of images in test set: {sample_sub_df["ImageId"].nunique()}')

### Image size analysis in training dataset

In [ ]:
pd.DataFrame([train_df['Height'].describe(), train_df['Width'].describe()]).T.loc[['max', 'min', 'mean']]

### Height and Width destribution of training images

In [ ]:
image_shape_df = train_df.groupby("ImageId")["Height", "Width"].first()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
ax1.hist(image_shape_df['Height'], bins=100)
ax1.set_title("Height distribution")
ax2.hist(image_shape_df['Width'], bins=100)
ax2.set_title("Width distribution")
plt.show()

### Image with minimum height

In [ ]:
x=list(set(train_df[train_df['Height'] == train_df['Height'].min()]['ImageId']))

In [ ]:
len(x)

In [ ]:
plt.figure(figsize = (70,7))
min_height = list(set(train_df[train_df['Height'] == train_df['Height'].min()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{min_height}.jpg'))
plt.grid(False)
plt.show()

### Image with maximum height

In [ ]:
plt.figure(figsize = (70,7))
max_height = list(set(train_df[train_df['Height'] == train_df['Height'].max()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{max_height}.jpg'))
plt.grid(False)
plt.show()

### Image with minimum width

In [ ]:
plt.figure(figsize = (70,7))
min_width = list(set(train_df[train_df['Width'] == train_df['Width'].min()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{min_width}.jpg'))
plt.grid(False)
plt.show()

### Image with maximum width

In [ ]:
plt.figure(figsize = (70,7))
max_width = list(set(train_df[train_df['Width'] == train_df['Width'].max()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{max_width}.jpg'))
plt.grid(False)
plt.show()

In [ ]:
area_df = pd.DataFrame()
area_df['ImageId'] = train_df['ImageId']
area_df['area'] = train_df['Height'] * train_df['Width']
min_area = list(set(area_df[area_df['area'] == area_df['area'].min()]['ImageId']))[0]
max_area = list(set(area_df[area_df['area'] == area_df['area'].max()]['ImageId']))[0]

### Image with minimum area

In [ ]:
plt.figure(figsize = (70,7))
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{min_area}.jpg'))
plt.grid(False)
plt.show()

### Image with maximum area

In [ ]:
plt.figure(figsize = (70,7))
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{max_area}.jpg'))
plt.grid(False)
plt.show()

## Details about Classes and Attributes

In [ ]:
num_classes = len(label_desc['categories'])
num_attributes = len(label_desc['attributes'])
print(f'Total # of classes: {num_classes}')
print(f'Total # of attributes: {num_attributes}')

In [ ]:
categories_df = pd.DataFrame(label_desc['categories'])
attributes_df = pd.DataFrame(label_desc['attributes'])

In [ ]:
pd.set_option('display.max_rows', 300)
attributes_df

# ## Plotting a few training images without any masks

In [ ]:
def plot_images(size=12, figsize=(12, 12)):
    # First get some images to be plotted
    image_ids = train_df['ImageId'].unique()[:12]
    images=[]
    
    for image in image_ids:
        images.append(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{image}.jpg'))
    
    # Plot images in groups of 4 images
    n_groups = 4
    
    count = 0
    for index in range(size // 4):
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
        for row in ax:
            for col in row:
                col.imshow(images[count])
                col.axis('off')
                count += 1
        plt.show()
    gc.collect()

In [ ]:
plot_images()

## Plotting a few images with given segments

In [ ]:
def create_mask(size):
    image_ids = train_df['ImageId'].unique()[:size]
    images_meta=[]

    for image_id in image_ids:
        img = mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{image_id}.jpg')
        images_meta.append({
            'image': img,
            'shape': img.shape,
            'encoded_pixels': train_df[train_df['ImageId'] == image_id]['EncodedPixels'],
            'class_ids':  train_df[train_df['ImageId'] == image_id]['ClassId']
        })

    masks = []
    for image in images_meta:
        shape = image.get('shape')
        encoded_pixels = list(image.get('encoded_pixels'))
        class_ids = list(image.get('class_ids'))
        
        # Initialize numpy array with shape same as image size
        height, width = shape[:2]
        mask = np.zeros((height, width)).reshape(-1)
        
        # Iterate over encoded pixels and create mask
        for segment, (pixel_str, class_id) in enumerate(zip(encoded_pixels, class_ids)):
            splitted_pixels = list(map(int, pixel_str.split()))
            pixel_starts = splitted_pixels[::2]
            run_lengths = splitted_pixels[1::2]
            assert max(pixel_starts) < mask.shape[0]
            for pixel_start, run_length in zip(pixel_starts, run_lengths):
                pixel_start = int(pixel_start) - 1
                run_length = int(run_length)
                mask[pixel_start:pixel_start+run_length] = 255 - class_id * 4
        masks.append(mask.reshape((height, width), order='F'))  # https://stackoverflow.com/questions/45973722/how-does-numpy-reshape-with-order-f-work
    return masks, images_meta

In [ ]:
def plot_segmented_images(size=12, figsize=(14, 14)):
    # First create masks from given segments
    masks, images_meta = create_mask(size)
    
    # Plot images in groups of 4 images
    n_groups = 4
    count = 0
    for index in range(size // 4):
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
        for row in ax:
            for col in row:
                col.imshow(images_meta[count]['image'])
                col.imshow(masks[count], alpha=0.75)
                col.axis('off')
                count += 1
        plt.show()
    gc.collect()

In [ ]:
plot_segmented_images()

In [ ]:
max

## Analysing Categories and Attributes

In [ ]:
categories_df = pd.DataFrame(label_desc.get('categories'))
attributes_df = pd.DataFrame(label_desc.get('attributes'))

In [ ]:
print(f'# of categories: {len(categories_df)}')
print(f'# of attributes: {len(attributes_df)}')

So there are 46 categories (classes) and 294 attributes. Let's see some of the categories and attributes

In [ ]:
categories_df.head()

In [ ]:
attributes_df.head()

In [ ]:
category_map, attribute_map = {}, {}
for cat in label_desc.get('categories'):
    category_map[cat.get('id')] = cat.get('name')
for attr in label_desc.get('attributes'):
    attribute_map[attr.get('id')] = attr.get('name')

In [ ]:
label_desc['categories']

In [ ]:
#seperated by a comma


def extract(x):
    i=0
    num=""
    extract_list=[]

    while(i<=len(x)):
        if(i==len(x)):
            extract_list.append(num)
            break
        if(x[i]!=','):
            num=num+x[i]
            i=i+1
        else:
            extract_list.append(num)
            i=i+1
            num=""
    return extract_list


In [ ]:
attributes_description_list=[]

for i in range(0,train_df.shape[0]):
    attributes_description_list.append(extract (str(train_df.loc[i,'AttributesIds'])))

In [ ]:
attributes_description_list

In [ ]:
train_df['Attributes_Values']=attributes_description_list

In [ ]:
attributes_list=[]


In [ ]:
label_desc['attributes'][295-47]

In [ ]:
attributes_sub_list=[]
attributes_list=[]

In [ ]:
for i in range(0, train_df.shape[0]):
    for j in train_df.loc[i,'Attributes_Values']:

        if j=='nan':
            attributes_sub_list.append('nan')
            
        elif int(j)>234:
            attributes_sub_list.append(label_desc['attributes'][int(j)-47]['name'])
        else:
            attributes_sub_list.append(label_desc['attributes'][int(j)]['name'])
    attributes_list.append(attributes_sub_list)
    attributes_sub_list=[]

In [ ]:
attributes_list


We are trying to see the variation of attributes per class ID

In [ ]:
train_df['train_attributes_value']=attributes_list

In [ ]:
train_df_cat_2=train_df[train_df['ClassId']==0]

In [ ]:
train_df_cat_2

**So we are trying to see for a particular class what is the various attributes value**

In [ ]:
train_df_cat_2.iloc[0]['train_attributes_value']

In [ ]:
all_attrubutes_class0_list=[]

Basically I am checking how many unique attributes are there in a class

In [ ]:

for i in range(0,train_df_cat_2.shape[0]):
#     print(len(train_df_cat_2.iloc[i]['train_attributes_value']))
    for j in train_df_cat_2.iloc[i]['train_attributes_value']:
        all_attrubutes_class0_list.append(j)


In [ ]:
x=set(all_attrubutes_class0_list)

In [ ]:
x=list(x)

In [ ]:
for i in x:
    print(i)

In [ ]:
class_attribute = np.zeros((47,341),dtype='int64')
clz_attrid2idx = [[] for _ in range(46)]
class_attribute.shape

For all the classe what is the distribution

In [ ]:
for c,i in zip(train_df.ClassId,train_df.Attributes_Values):
    for a in i:
         if a!='nan':
             class_attribute[int(c),int(a)] = int(1)

In [ ]:
clz_attr_num = class_attribute.sum(axis=1)

In [ ]:
clz_attr_num

In [ ]:
label_desc['attributes'][293]

In [ ]:
train_df

In [ ]:
def rle_to_mask(rle_string,height,width):
    rows, cols = height, width
    if rle_string == -1:
        return np.zeros((height, width))
    else:
        rleNumbers = [int(numstring) for numstring in rle_string.split(' ')]
        rlePairs = np.array(rleNumbers).reshape(-1,2)
        img = np.zeros(rows*cols,dtype=np.uint8)
        for index,length in rlePairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img

In [ ]:
df = train_df.iloc[1]

In [ ]:
mask = rle_to_mask(df.EncodedPixels,df.Height,df.Width)
imag = cv2.imread("../input/imaterialist-fashion-2020-fgvc7/train/"+str(df.ImageId)+".jpg")
imag[mask==0] = 255

In [ ]:
imag

In [ ]:
where = np.where(imag < 255)
if len(where[0]) > 0 and len(where[1]) > 0:
    y1= min(where[0])
    y2=max(where[0])
    x1=min(where[1])
    x2=max(where[1])
    
plt.imshow(imag[y1:y2,x1:x2])

In [ ]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
train_df['ClassId'] = train_df['ClassId'].map(category_map)
train_df['ClassId'] = train_df['ClassId'].astype('category')

### Let's see the class wise distribution of segments in training dataset

In [ ]:
sns.set(style='darkgrid')
fig, ax = plt.subplots(figsize = (10,10))
sns.countplot(y='ClassId',data=train_df , ax=ax, order = train_df['ClassId'].value_counts().index)
fig.show()

### Now let's visualize an image with all its classes and attributes

In [ ]:
IMAGE_ID = '000b3ec2c6eaffb491a5abb72c2e3e26'

In [ ]:
# Get the an image id given in the training set for visualization
vis_df = train_df[train_df['ImageId'] == IMAGE_ID]
vis_df['ClassId'] = vis_df['ClassId'].cat.codes
vis_df = vis_df.reset_index(drop=True)
vis_df

From above table, this image has 8 segmentes and a few attributes. Let's visualize all of them!

## Let's first the plot the plain image

In [ ]:
plt.figure(figsize = (110,11))
image = mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{IMAGE_ID}.jpg')
plt.grid(False)
plt.imshow(image)
plt.plot()

In [ ]:
train_df[train_df['ImageId'] == IMAGE_ID]

## Now let's plot each segment in a separate image

In [ ]:
segments = list(vis_df['EncodedPixels'])
class_ids = list(vis_df['ClassId'])
masks = []
for segment, class_id in zip(segments, class_ids):
    
    height = vis_df['Height'][0]
    width = vis_df['Width'][0]
    # Initialize empty mask
    mask = np.zeros((height, width)).reshape(-1)
    
    # Iterate over encoded pixels and create mask
    splitted_pixels = list(map(int, segment.split()))
    pixel_starts = splitted_pixels[::2]
    run_lengths = splitted_pixels[1::2]
    assert max(pixel_starts) < mask.shape[0]
    for pixel_start, run_length in zip(pixel_starts, run_lengths):
        pixel_start = int(pixel_start) - 1
        run_length = int(run_length)
        mask[pixel_start:pixel_start+run_length] = 255 - class_id * 4

    mask = mask.reshape((height, width), order='F')
    masks.append(mask)

In [ ]:
def plot_individual_segment(*masks, image, figsize=(110, 11)):
    plt.figure(figsize = figsize)
    plt.imshow(image)
    for mask in masks:
        plt.imshow(mask, alpha=0.6)
    plt.axis('off')
    plt.show()

## Plotting 1st Segment: ClassId: "Shoe" and no attributes 

In [ ]:
plot_individual_segment(masks[0], image=image)

## Plotting 2nd Segment: ClassId: "shoe"

In [ ]:
plot_individual_segment(masks[1], image=image)

## Plotting 3rd Segment with ClassId: "pants"

In [ ]:
plot_individual_segment(masks[2], image=image)

* ## Plotting 4th Segment with ClassId: "top, t-shirt, sweatshirt"

In [ ]:
plot_individual_segment(masks[3], image=image)

## Plotting 5th Segment with ClassId: "pocket"

In [ ]:
plot_individual_segment(masks[4], image=image)

## Plotting 6th Segment with ClassId: "sleeve"

In [ ]:
plot_individual_segment(masks[5], image=image)

## Plotting 7th Segment with ClassId: "sleeve"

In [ ]:
plot_individual_segment(masks[6], image=image)

## Plotting 8th segment with Class "neckline"

In [ ]:
plot_individual_segment(masks[6], image=image)

Some of the segments have no attributes. Let's check how many such segment exists in training dataset.

In [ ]:
print(f'Segments that do not have attributes: {train_df["AttributesIds"].isna().sum()/len(train_df) * 100} %')

In [ ]:
train_df['ClassId'] = train_df['ClassId'].cat.codes

In [ ]:
train_df_classID_2=train_df[train_df['ClassId']==2]

In [ ]:
train_df_classID_2=train_df_classID_2.dropna(subset=['AttributesIds'])

In [ ]:
train_df_classID_2['AttributesIds']=train_df_classID_2['AttributesIds'].astype('int64')

In [ ]:
train_df_classID_2=train_df[train_df['ClassId']==2]

In [ ]:
train_df

In [ ]:

for i in range (2,3):
    if train_df['AttributesIds'][i]=='163':
        print("hello")
        

In [ ]:
train_df_classID_2.loc[75:,'AttributesIds']

In [ ]:
sns.set(style='darkgrid')
fig, ax = plt.subplots(figsize = (10,10))
sns.countplot(y='AttributesIds',data=train_df_classID_2, ax=ax, order = train_df_classID_2['AttributesIds'].value_counts().index)
fig.show()

Matching class id with attributes ID to understand possible attributes(text) 

In [ ]:
label_desc['attributes']


Let's check of missing values in training dataset for columns other than "AttributeIds"

In [ ]:
train_df[['ImageId', 'EncodedPixels', 'Height', 'Width', 'ClassId']].isna().sum()

## Data Preparation and modeling

In [ ]:
train_df

In [ ]:
train_df['ClassId'] = train_df['ClassId'].cat.codes

In [ ]:
train_df

1 Drop attributeIds for simplicity for now. TODO: Need to take this in consideration once the basic model is ready with ClassId

In [ ]:
train_df = train_df.drop('AttributesIds', axis=1)

In [ ]:
image_df = train_df.groupby('ImageId')['EncodedPixels', 'ClassId'].agg(lambda x: list(x))
size_df = train_df.groupby('ImageId')['Height', 'Width'].mean()
image_df = image_df.join(size_df, on='ImageId')

print("Total images: ", len(image_df))
image_df.head()

Reference: https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb

In [ ]:
# !pip install tensorflow==1.15

In [ ]:
# import os
# from pathlib import Path
# !git clone https://www.github.com/matterport/Mask_RCNN.git
# os.chdir('Mask_RCNN')

# !rm -rf .git # to prevent an error when the kernel is committed
# !rm -rf images assets # to prevent displaying images at the bottom of a kernel

In [ ]:
# !wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
# !ls -lh mask_rcnn_coco.h5

# COCO_WEIGHTS_PATH = 'mask_rcnn_coco.h5'

In [ ]:
DATA_DIR = Path('/kaggle/input/imaterialist-fashion-2020-fgvc7')
ROOT_DIR = Path('/kaggle/working')

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
class FashionConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "class"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + len(categories_df)  # background + 46 classes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = FashionConfig()
config.display()

In [ ]:
class FashionDataset(utils.Dataset):
    def __init__(self, df):
        super().__init__(self)
        
        self.IMAGE_SIZE = 256
        
        # Add classes
        for cat in label_desc['categories']:
            self.add_class('fashion', cat.get('id'), cat.get('name'))
        
        # Add images
        for i, row in df.iterrows():
            self.add_image("fashion", 
                           image_id=row.name, 
                           path=str(DATA_DIR/'train'/row.name) + '.jpg', 
                           labels=row['ClassId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])
            
    def _resize_image(self, image_path):
        print(image_path)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.IMAGE_SIZE, self.IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
        return img
        
    def load_image(self, image_id):
        return self._resize_image(self.image_info[image_id]['path'])
    
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [x for x in info['labels']]
    
    def load_mask(self, image_id):
        info = self.image_info[image_id]
                
        mask = np.zeros((self.IMAGE_SIZE, self.IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []
        
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8)
            annotation = [int(x) for x in annotation.split(' ')]
            
            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1

            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (self.IMAGE_SIZE, self.IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
            
        return mask, np.array(labels)

In [ ]:
dataset = FashionDataset(image_df)
dataset.prepare()

for i in range(6):
    image_id = random.choice(dataset.image_ids)
    print(dataset.image_reference(image_id))
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=4)

In [ ]:
# This code partially supports k-fold training, 
# you can specify the fold to train and the total number of folds here
FOLD = 0
N_FOLDS = 5

kf = KFold(n_splits=N_FOLDS, random_state=42, shuffle=True)
splits = kf.split(image_df) # ideally, this should be multilabel stratification

def get_fold():    
    for i, (train_index, valid_index) in enumerate(splits):
        if i == FOLD:
            return image_df.iloc[train_index], image_df.iloc[valid_index]
        
train_df, valid_df = get_fold()

train_dataset = FashionDataset(train_df)
train_dataset.prepare()

valid_dataset = FashionDataset(valid_df)
valid_dataset.prepare()

## Training

In [ ]:
# Note that any hyperparameters here, such as LR, may still not be optimal
LR = 1e-4
EPOCHS = [2, 6, 8]

import warnings 
warnings.filterwarnings("ignore")

Let's load the COCO dataset weights to our Model.

In [ ]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

# Load weights trained on MS COCO, but skip layers that
# are different due to the different number of classes
# See README for instructions to download the COCO weights
model.load_weights(COCO_MODEL_PATH, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                            "mrcnn_bbox", "mrcnn_mask"])

# In Progress... Stay Tuned!